<h1><center>Hyper parameter optimization</center></h1>

This is a short tutorial on how hyper-parameter optimization works with a small example as we go along. The contents of this tutorial is taken from:

1. https://www.cs.ubc.ca/~nando/papers/bayopt.pdf
2. https://arxiv.org/pdf/1812.06855.pdf

The data for the model is taken from kaggle churn prediction competition. We will use xgboost to predict if given users will churn or not, based on the information collected from them by the telecom company.

<h2>Definition of hyper-parameters</h2>

In a machine learning algorithm, Hyper-parameters are parameters to the model. These parameters are external to the model and their values cannot be estimated from the data given to the model. Th parameters which are internal to the model and that can be estimated from data are called model parameters. Example: In a lasso regression, we can set the weight of the L1 penalty.

In Lasso, the coefficients for the predictor/independent variables are model parameters, While the coefficient for L1 penalty (that is fed into the lasso regression. is the hyper-parameter).

As the model becomes complex, the number of hyper-parameters increases and it becomes difficult to find the best set of hyper-parameters. This is also valid for very complex optimization problems (other than ML models) that takes too much time such that all the different combinations of hyper-parameters can be searched using brute force to find the best set of options.

It used to be called grad student search. It was joke that grad students need to check all the different combinations of hyper-parameters. There are three methods to do that: 

1. <b>using brute force to search all the possible combinations</b>. In this method, the number of options increase exponentially and becomes impossible to search all.
2. <b>using random search</b> to search for different combinations of hyper-parameters. Random search has a higher chances of accidentally stumbling upon a better set of hyper-parameters as it searches in the space randomly. However, it does not use the information it has learned while searching.
3. <b>intelligently searching</b> for combination of hyper-parameters. This is the topic of this tutorial where we use bayesian methods to intelligently search for hyper-parameters, by considering search regions already explored.

If you haven't, please see the <a href='https://www.youtube.com/watch?v=HZGCoVF3YvM'>this</a> video on bayes theorem before proceeding.

In [1]:
# reading the data -----
# the data has multiple columns and one column for Y (churn)
# you need to install the libraries if not installed e.g xgboost

import pandas as pd
import numpy  as np
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score

data = pd.read_csv('data_new.csv')

In [3]:
X = pd.DataFrame.copy(data)
del X['customerID']
del X['Churn']
del X['TotalCharges']

Y = data['Churn']

<h2>XGBoost model</h2>

In [19]:
'''
XGBoost has following python API with so many different hyper-parameters. The number of hyper-parameters increases 
as the models become more and more complex, e.g., neural networks

class xgboost.XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, objective='reg:squarederror', 
      booster='gbtree', tree_method='auto', n_jobs=1, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, 
      colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, 
      base_score=0.5, random_state=0, missing=None, num_parallel_tree=1, importance_type='gain', **kwargs)
      
''';

In [29]:
global X_train,X_test,X_val, Y_train, Y_test, Y_val

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)           # train and test
X_val, X_test, Y_val, Y_test     = train_test_split(X_test, Y_test, test_size=0.2, random_state=42) # test and validation
print(X.shape,X_train.shape,X_test.shape,Y_train.shape,Y_test.shape, X_val.shape, Y_val.shape)

(7043, 25) (5634, 25) (282, 25) (5634,) (282,) (1127, 25) (1127,)


In [21]:
# this is how parameters can be set
parameter = {'max_depth':5, 'n_estimators':100, 'eta':1, 'objective':'binary:logistic','min_child_weight':1}

model = xgb.XGBClassifier(**parameter)
model.fit(X_train,Y_train)

y_pred      = model.predict(X_test)                 # predicting on the test data set
predictions = [round(value) for value in y_pred]    # converting probability to binary
accuracy    = accuracy_score(Y_test, predictions)   # finding the accuracy
print(accuracy)

# WE WILL COME BACK TO THIS LATER, FIRST WE GO THROUGH WHAT IS BAYESIAN HYPER_PARAMETER OPTIMIZATION

0.8014184397163121


<h2>Bayesian Hyper-parameter Optimization</h2>

We first build a framework for bayesian statistics. There has been some differences between frequentists and bayesian statisticians. Bayesian statistics start building a model based on a <b>prior</b> belief. Once they observe the <b> data</b>, they re-evaluate their belief and update thier belief and call it <b>posterior</b>.  

<h3>Bayes Theorem</h3>

Bayes theorem forms the basis for bayesian statistics. Before reading further, please see <a href="https://www.youtube.com/watch?v=HZGCoVF3YvM">this</a> video. It visually explains what bayes theorem is and how the formula for the bayes theorem is derived. In the bayesian hyper-parameter optimization, we update our belief on the area we have already explored. If it is worth exploring in that region, we will explore more in that region (exploitation) otherwise search for a new region (exploration).

<img src='Bayes.png' width=600></image>


<center>Figure 1. Bayes Theorem</center>

<h3>Idea of Bayesian Optimization</h3>

We build a prior for the objective function as how it changes with the changes in the value of x. As we explore and try new and new values of x, we update our prior. We find the next point to explore using a surrogate function (more on it later). The surrogate function uses a trade-off between exploitation and exploration to find the next x to explore.
<img src='Optimization.png' width=700></image>

<center>Figure 2. Bayes idea in optimization</center>

An example of how to find the next point to explore is shown in the figure below. Based on our belief of how the objective function changes (or is related with) x, we try to find the next point (based on a surrogate function that is itself a function of mean and variance of our belief). We explore the next point and update our belief. 

1. <b>Exploration</b> If mean is higher, there is a high chance that it is closer to the optimal value
2. <b>Exploitation</b> If the variance is high, we do not know much about that region (there could be the optimal value)
<img src='Idea.png' width = 600></image>

<center>Figure 3. Idea for bayesian optimization procedure</center>

<h3>Gaussian Process</h3>

Gaussian process is the most common and most important topic in statistics and is a base for a lot of ML algorithms. For a background on Gaussian processes, please see the <a href="https://www.youtube.com/watch?v=4vGiHC35j9s">video</a> by Dr. Freitas. A Gaussian process is a process in which there is a sequence of data points that come from a multi-variate normal distribution. If all the other dimension are kept at a fixed value (except 1), it is called marginal distribution and it is also normally distributed.

Example, think of a hill as a bivariate gaussian distribution. In gaussian process, we will get sequence of data points (x,y) from this hill. We drop a big vertical axe from the sky and it chops off a part of the hill and separates it. If we stand in front of the chopped hill, we will see a normal distribution. This is called conditioning, P(x1|x2) , which is normally distributed).

<b>Smoothness</b> For small change in x, we observe a small change in y.

<b>How do we make use of gaussian process in the figures above?</b>

We use the concept behind gaussian distribution and smoothness to find the expected mean and variance of a new location. This is explained further in the Figure blow. We define the correlation between new points using a distance based metric. It ensures that two points that are close together have larger correlation. Also, variance is low for a point that is closer to the known points but higher for farther away points. By calculating conditional, we can estimate the mean and variance of any point in the search region. 

<img src='Gaussian.png' width = 600></image>

<center>Figure 4. Gaussian Process</center>

<h3>Surrogate utility Function</h3>

When we start looking for the best x, we start by randomly checking 4-5 points (to build our prior). Now, to go for the next point, we look for a point that can provide us maximum utility. This maximum utility is calculated using a surrogate function. It is also known as <b>acquisition function or expected improvement function</b> as shown in Figure 3. For any new point, the value for the function is drawn using the procedure shown in Figure 4. After this value is drawn for mean and variance, then acquisition function value is calculated and the x which maximizes the acquisition function is selected. <b>Note:</b> How we select the correlation (similarity function) is important (please go through the paper to read on finer details as we will be using python library bayesOpt). 

The example shown in Figure 4 considers just one step look ahead (select just the next x). However, this could be myopic and algorithm can select the best value of x looking two step ahead. But expected improvement function can be used as an alternative (and we do not need to do dynamic programming for more than 1 step look ahead). The utility function (acquisition function/ expected improvement function) is shown in Figure 5.

<img src='Utility.png' width = 800></image>

<center>Figure 5. Expected utility function: How it works</center>


<h3>How do we find x from surrogate model?</h3>

We know that for any x, how to calculate the expected improvement function. We can sample any x and get the utility function value. The surrogate problem is not a hard problem and it can be sampled very cheaply (as surrogate gate problem structure is very easy to compute). In the paper, they mentioned they use DIRECT method : DIvide feasible space into RECTangles. This is a simple sampling method, however it achieves similar performance as more complex optimization procedures e.g. quadratic programming, branch and bound, etc.

<h3>A real life example</h3>

Lets take a look at an example using one-dimension x variable. Consider we want to drill in a 1000 km stretch of land (marking from 0 to 1000 km) to find oil reserves. Since drilling is expensive, we cannot drill in a large numbers to find oil. Thus we have to intelligently drill to find a reasonably good area of oil with minimum number of drills. Since we do not know the function as how close the oil is (or how much oil is in that pocket), that function is unknown but once we drill a place, we know how much oil there was. 

The assumption is still (kind of) valid that on changing x by small amount, y will not change drastically (smoothness). That is if we drill in a marking of 450 km, we will not gain much information (variance is low) and increment (smoothness) on drilling in 451 km marking. Also, as we move away and away from the regions where we have drilled before, the unknown (or variance : measure of uncertainty) increases. There is also a scope that there is more oil in that unexplored/unknown region. Thus, drilling operation needs careful selection of location based on information from past drilling. We have to balance exploration and exploitation. If there is huge oil when drilled in the marking for 800 km and 850 km, we must check for oil in 825 km (exploitation: searching in a region that has provided good results). Also, we must not leave a large stretch. If we do not check from 100 km to 200 km marking, we might miss out. Thus we should explore more too (exploration). 

<h3>BayesOpt library in Python</h3>

Now that we know what we are doing when we optimizing hyper-parameters, we will revisit the problem in hand of predicting which customer will churn. We will use f-score to measure the performance of the algorithm (accuracy would be a bad measure as the data is imbalanced).

In [74]:
# hyperOpt library, if not installed, please install it

from hyperopt import hp
from hyperopt import STATUS_OK, fmin

# optimization algorithm
from hyperopt import tpe
tpe_algorithm = tpe.suggest

# generating trials using bayes
from hyperopt import Trials
bayes_trials = Trials()

In [75]:
# first create an objective function to calculate and return the f-score for validation set

def objective(parameter):
    model       = xgb.XGBClassifier(**parameter).fit(X_train,Y_train)
    y_pred      = model.predict(X_val)                  # predicting on the test data set
    predictions = [round(value) for value in y_pred]    # converting probability to binary
    fscore      = f1_score(y_pred, Y_val) 
    
    return (-fscore)

In [80]:
# suppose we want to find the best of the following parameters
# we define the search space where we want the algorithm to search

search_space = {
    'n_estimators' : hp.choice('n_estimators', np.arange(30, 150,dtype=int)),  # 30:start, 50:end, 1:step size
    'max_depth'    : hp.choice('max_depth', np.arange(3, 10,dtype=int)),
    'min_child_weight': hp.uniform('min_child_samples', 1, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'learning_rate': hp.loguniform('learning_rate',np.log(0.005), np.log(0.2))
               }

In [81]:
# find the best parameters

number_of_iteration = 25
best = fmin(fn = objective, space = search_space, algo = tpe.suggest, 
            max_evals = number_of_iteration, trials = bayes_trials)

100%|███████████████████████████████████████████████| 25/25 [00:56<00:00,  1.61s/trial, best loss: -0.6413223140495868]


In [82]:
# printing the best parameters obtained
best

{'learning_rate': 0.005061719499982551,
 'max_depth': 2,
 'min_child_samples': 4.881818170768246,
 'n_estimators': 71,
 'reg_alpha': 0.5928909152872696,
 'reg_lambda': 0.12798279641549615}

<h2>Bayesian Optimization in alphaGo</h2>

We have all heard about alphaGo. Bayesian optimization can be used to find the best set of hyper-parameters. A portion of the abstract from the second paper mentioned in the start of this tutorial:

<i>During the development of AlphaGo, its many hyper-parameters were tuned with Bayesian optimization multiple times. This automatic tuning process resulted in substantial improvements in playing strength. For example, prior to the match with Lee Sedol, we tuned the latest AlphaGo agent and <b>this improved its win-rate from 50% to 66.5% in self-play games. This tuned version was deployed in the ﬁnal match. Of course, since we tuned AlphaGo many times during its development cycle,the compounded contribution was even higher than this percentage</b> </i>

THey further mention that bayesian optimization could help them in understanding the individual effect of each of the parameters. They used bayesian optimization in game playing part of alphaGo. They did not use it for neural network part of the alphaGo. Please go through the paper as how and where they implemented bayesian optimization procedure.

<h2>Other Uses of this Approach</h2>

It can be certainly used in any problem which is complicated and takes a long time to reach the solution. One such problems is simulation. Simulations, for example, a manufacturing facility or electricity grid, has to run for long time period, e.g. simulate the system for 10-20 years. As we increase the model fidelity, the simulation becomes slower (for example, we run our power grid simulation for every second rather than every hour). 

It can also be used for problems that are not only computationally complex but are expensive in general. For example, drug trials as research and development is very expensive to carry out (before making some changes in the drug).